### 시상검증용 스크립트(곰스)

In [ ]:
import pandas as pd
import numpy as np
import duckdb
from openpyxl import load_workbook

repl_dict = {True:'생보',False:'손보'}
keywords = ['생명','라이프']
def bojong(co_name): #회사명으로 생,손보 구분
    return repl_dict.get(any( k in co_name for k in keywords))

#### goms산출 파일 및 엑셀 파일 read

In [ ]:
path='E:/글로벌백업(180329)/04.수수료작업/202504시상/'
# f_name= 'GOMS수입명세-검증.xlsx'
f_name= '2025년 4월 업적_수입원장.xlsx'
sht = '건별'
in_df = pd.read_excel(path+f_name, sheet_name=sht)
in_df = in_df[~in_df['보험사'].isna()]
in_df['구분'] = in_df['보험사'].apply(lambda x: bojong(x))
if '수입원장' in f_name:
    vals = ['(지급/환수)시상','리스크공제']
    in_df.rename(columns={'적용일':'적용일자'},inplace=True)
else:
    vals = ['지급','환수','리스크공제']

in_sum = pd.pivot_table(in_df, values=vals, index=['구분','적용일자','보험사'],
            columns=['시상구분','시상분류'], aggfunc=np.sum, fill_value=0, margins=False, sort=False).reset_index()
in_sum.to_clipboard(index=False)
in_sum.head(1)
print('수입원장 집계 복사완료')

In [ ]:
f_name= '2025년 4월 업적_수입원장.xlsx'
sht = '항목별'
in_df2 = pd.read_excel(path+f_name, sheet_name=sht)
in_df2 = in_df2[~in_df2['보험사'].isna()]
in_df2['구분'] = in_df2['보험사'].apply(lambda x: bojong(x))
vals = '(지급/환수)시상금'
in_df2.rename(columns={'적용일':'적용일자'},inplace=True)
in_sum2 = pd.pivot_table(in_df2[in_df2['적용일자'].isin(['말일자','10일자'])],
                         values=vals, index=['구분','시상구분','적용일자','보험사'],
            columns=['시상분류'], aggfunc=np.sum, fill_value=0, margins=False, sort=False).reset_index()
in_sum2.to_clipboard(index=False)
in_sum2.head(1)
print('수입원장 항목별 집계 복사완료')

In [ ]:
in_sum2

In [ ]:
path='E:/글로벌백업(180329)/04.수수료작업/202504시상/'
f_name= 'GOMS지급명세-검증.xlsx'
pay_df = pd.read_excel(path+f_name, sheet_name='지급내역상세조회_상세내역조회', dtype={'증권번호': 'object'})
pay_df = pay_df[~pay_df['보험사'].isna()]
pay_df['구분'] = pay_df['보험사'].apply(lambda x: bojong(x))
pay_df['3단계(지점)시상'] = pay_df['2단계(직할지점)시상'] + pay_df['3단계(지점)시상']

pay_sum = pd.pivot_table(pay_df, values=['FC시상계','OR시상계','3단계(지점)시상','5단계(총괄)시상'],
                index=['시상구분','구분','보험사'],
                columns=['시상분류'], aggfunc=np.sum, fill_value=0, margins=False,
                          sort=False).reset_index()
pay_sum.to_clipboard(index=False)
pay_sum.head(1)
print('지급원장 집계 복사완료')

In [ ]:
pay_sum.to_clipboard(index=False)

#### 지급명세와 수입명세 데이터 불일치건 찾기

In [ ]:
''' 사별로 지급과 수입을 비교한다 --건별시상이므로 GA시상금 비교에 사용'''
#지급내역
pay_corp_sum = pay_df.groupby(['보험사','시상분류'])[['FC시상계','OR시상계']].sum().reset_index()
pay_corp_sum['계'] = pay_corp_sum['FC시상계'] + pay_corp_sum['OR시상계']
pay_corp_sum.head(1)
#수입내역
in_corp_sum = in_df.groupby(['보험사','시상분류'])[['(지급/환수)시상','리스크공제']].sum().reset_index()
in_corp_sum['계'] = in_corp_sum['(지급/환수)시상'] + in_corp_sum['리스크공제']
in_corp_sum.head(1)
#비교
sum_compare = pd.merge(in_corp_sum, pay_corp_sum, how='outer', on=['보험사','시상분류']).fillna(0)
sum_compare['차이'] = sum_compare['계_x'] - sum_compare['계_y']

''' =====> 차이건중 FC시상이 있으면 확인 필요 '''
sum_compare[sum_compare['차이']!=0]


In [ ]:
'''건별 비교을 위한 준비 '''
pay1_df = in_df
pay1_df = pay1_df[~pay1_df['보험사'].isna()]
pay1_df['구분'] = pay1_df['보험사'].apply(lambda x: bojong(x))
pay1_df.columns
pay1_df['(시상금/리스크)합계 시상금'] = pay1_df['(지급/환수)시상'] + pay1_df['리스크공제']

시상구분='보험사'
시상분류=['GA시상','총괄시상']
grp_col1 = ['구분','보험사','지점', '증권번호']
pay_pol = pay_df[(pay_df['시상구분']==시상구분) & (pay_df['시상분류'].isin(시상분류))]. \
                 groupby(grp_col1)[['OR시상계','3단계(지점)시상','5단계(총괄)시상']]. \
                 agg('sum').reset_index()
pay1_pol = pay1_df[(pay1_df['시상구분']==시상구분) & (pay1_df['시상분류'].isin(시상분류))]. \
                 groupby(grp_col1)[['(지급/환수)시상','리스크공제','(시상금/리스크)합계 시상금']]. \
                 agg('sum').reset_index()
mer1_pol = pd.merge(pay1_pol, pay_pol, how='outer', on='증권번호')
mer1_pol.fillna(0, inplace=True)
mer1_pol['차이'] = mer1_pol['(시상금/리스크)합계 시상금'] - mer1_pol['OR시상계']
compare1_df = mer1_pol[mer1_pol['차이']!=0] 
compare1_df.drop(columns=['지점_y','구분_y'], inplace=True)
print('아래 실행하여 불일치건 찾기')

In [ ]:
검증보종 = None # '손보' # None
검증회사 = '삼성화재'
# compare_df = compare_df.fillna('')
if 검증보종=="생보" or 검증보종=="손보":
    dd = compare1_df[(compare1_df ['구분_x']==검증보종) & (abs(compare1_df ['차이'])>2)]
elif 검증회사:
    dd = compare1_df[(compare1_df ['보험사_x']==검증회사) & (abs(compare1_df ['차이'])>2)]
dd_final = dd[['보험사_x','지점_x','증권번호','(지급/환수)시상','리스크공제','(시상금/리스크)합계 시상금'
               ,'OR시상계' ,'차이']]
dd_final

### 지급명세와 총괄기초 데이터 불일치건 찾기

In [ ]:
f_name= 'GOMS지급명세-검증.xlsx'
pay1_df = pd.read_excel(path+f_name, sheet_name='총괄지급용기초데이터', dtype={'증권번호': 'object'})
pay1_df = pay1_df[~pay1_df['보험사'].isna()]
pay1_df['구분'] = pay1_df['보험사'].apply(lambda x: bojong(x))
print('총괄지급명세')

In [ ]:
시상구분='보험사'
시상분류=['GA시상','총괄시상']
grp_col1 = ['구분','보험사','지점', '증권번호']
pay_pol = pay_df[(pay_df['시상구분']==시상구분) & (pay_df['시상분류'].isin(시상분류))]. \
                 groupby(grp_col1)[['OR시상계','3단계(지점)시상','5단계(총괄)시상']]. \
                 agg('sum').reset_index()
pay1_pol = pay1_df[(pay1_df['시상구분']==시상구분) & (pay1_df['시상분류'].isin(시상분류))]. \
                 groupby(grp_col1)[['(지급/환수)시상금','리스크공제','(시상금/리스크)합계 시상금']]. \
                 agg('sum').reset_index()
mer1_pol = pd.merge(pay1_pol, pay_pol, how='outer', on='증권번호')
mer1_pol.fillna(0, inplace=True)
mer1_pol['차이'] = mer1_pol['(시상금/리스크)합계 시상금'] - mer1_pol['OR시상계']
compare1_df = mer1_pol[mer1_pol['차이']!=0] 
compare1_df.drop(columns=['지점_y','구분_y'], inplace=True)
print('아래 실행하여 불일치건 찾기')

In [ ]:
검증보종 = None # '손보' # None
검증회사 = '삼성화재'
# compare_df = compare_df.fillna('')
if 검증보종=="생보" or 검증보종=="손보":
    dd = compare1_df[(compare1_df ['구분_x']==검증보종) & (abs(compare1_df ['차이'])>2)]
elif 검증회사:
    dd = compare1_df[(compare1_df ['보험사_x']==검증회사) & (abs(compare1_df ['차이'])>2)]
dd_final = dd[['보험사_x','지점_x','증권번호','(지급/환수)시상금','리스크공제','(시상금/리스크)합계 시상금'
               ,'OR시상계' ,'차이']]
dd_final

#### 엑셀 시상정산 읽기

In [ ]:
path='E:/글로벌백업(180329)/04.수수료작업/202503시상/'
f_name= '202503 KJ시상정산.xlsx'
#생보, 장기보험 건별 정산
cls_nm2 = ['지점','보험회사','증권번호','월납보험료','GLAD','지사시상']
xls_df = pd.concat([pd.read_excel(path+f_name, sheet_name='생보', dtype={'증권번호': 'object'})[cls_nm2],
                    pd.read_excel(path+f_name, sheet_name='손보', dtype={'증권번호': 'object'})[cls_nm2]])
xls_df['구분'] = xls_df['보험회사'].apply(lambda x: bojong(x))
xls_df['지사시상'] = [ round(x,0) for x in xls_df['지사시상']]

#### 비교검증(OR시상,지사시상)
생보는 총괄시상이 goms 지급내역에 0으로 산출되며, 손보는 리스크 자금 산출방법 차이가 남 

In [ ]:
## 특정회사 검증: 엑셀 vs goms (건별 지급시상금 산출)
grp_col1 = ['구분','보험사','지점', '증권번호']
grp_col2 = ['구분','보험회사','지점', '증권번호']
pay_pol = pay_df.groupby(grp_col1)[['OR시상계','3단계(지점)시상','5단계(총괄)시상']]. \
                 agg('sum').reset_index()
xls_pol = xls_df.groupby(grp_col2)[['GLAD','지사시상']].agg('sum').reset_index()
mer_pol = pd.merge(xls_pol, pay_pol, how='outer', on='증권번호')
#법인시상금 차이
mer_pol['차이'] = mer_pol['GLAD'] - mer_pol['OR시상계']
#지사 시상금 차이
mer_pol['지사차이'] = mer_pol['지사시상'] - mer_pol['3단계(지점)시상']
compare_df = mer_pol[mer_pol['차이']!=0] 

compare_df.drop(columns=['지점_y','구분_y'], inplace=True)

In [ ]:
''' 총액 검증 '''
검증보종 = '손보' #'손보' # None
# 검증회사 = 'DB생명'
# compare_df = compare_df.fillna('')
if 검증보종=="생보" or 검증보종=="손보":
    dd = compare_df[(compare_df ['구분_x']==검증보종) & (abs(compare_df ['차이'])>2)]
elif 검증회사:
    dd = compare_df[(compare_df ['보험회사']==검증회사) & (abs(compare_df ['차이'])>2)]
dd_final = dd[['보험회사','지점_x','증권번호','GLAD','OR시상계','차이']]
dd_final.to_clipboard()
dd_final

In [ ]:
''' 지사 검증 '''
검증보종 = '손보' # None
검증회사 = '삼성생명'
# compare_df = compare_df.fillna('')
if 검증보종=="생보" or 검증보종=="손보":
    dd = compare_df[(compare_df ['구분_x']==검증보종) & (abs(compare_df ['지사차이'])>2)]
elif 검증회사:
    dd = compare_df[(compare_df ['보험회사']==검증회사) & (abs(compare_df ['지사차이'])>2)]
dd_final = dd[['보험회사','지점_x','증권번호','GLAD','지사시상','3단계(지점)시상','5단계(총괄)시상','지사차이']]
dd_final.to_clipboard()
dd_final

In [ ]:
dd_final['지점_x'].unique()

In [ ]:
# 특정증번 비교
pl_no='52420154490000'
# 지급내역 산출
cls_nm= ['보험사', '증권번호', '시상구분', '시상분류', '지점',
         '보험료', 'FC시상계', 'OR시상계', '3단계(지점)시상', '5단계(총괄)시상']
pay_df[(pay_df['증권번호']==pl_no) & (pay_df['보험료']!=0)][cls_nm]

In [ ]:
# 수입내역
cls_nm3 = ['보험사', '증권번호', '시상구분', '시상분류', '지점',
         '보험료', '지급', '환수']
in_df[in_df['증권번호']==pl_no][cls_nm3]

#### 본사 시상 건별비교 (수입시상금 vs 지급시상금)

In [ ]:
cols = ['보험사','증권번호','보험료','지급','환수']
in_list = in_df[in_df['시상구분']=='본사'][cols]
in_list['FC시상'] = in_list['지급'] + in_list['환수']
in_list.rename({'FC시상계':'FC시상'}, inplace=True)

cols = ['보험사','증권번호','보험료','FC시상계']
pay_list = pay_df[pay_df['시상구분']=='본사'][cols]
pay_list.rename({'FC시상계':'FC시상'}, inplace=True)

In [ ]:
''' join 및 값 비교'''
df_ply = pd.merge(in_list, pay_list, how = 'outer',
            on=['증권번호'],indicator = True).reset_index().astype({"_merge": str}).fillna(0)
df_ply.drop(columns='index',inplace=True)

df_ply['증번비교']= ["일치" if x == 'both'  else ( '수입만 존재' if x == 'left_only' else "지급만 존재") 
                 for x in df_ply['_merge'] ]

df_ply['회사비교']= ["일치" if str(x) == str(y)  else ("불일치" if str(z)=='both' else "")
                 for x,y,z in zip(df_ply['보험사_x'],df_ply['보험사_y'],df_ply['_merge'])]

df_ply['시상금비교']= ["일치" if round(x) == round(y)  else ("불일치" if str(z)=='both' else "")
                 for x,y,z in zip(df_ply['FC시상'],df_ply['FC시상계'],df_ply['_merge'])]
exp  = f" (증번비교 != '일치') or (회사비교 != '일치') or (시상금비교 != '일치') "
df_error = df_ply.query(exp)
df_error.to_clipboard(index=False)